## Choose GPU

In [1]:
# From: https://github.com/keras-team/keras/issues/6031
import os
gpu_id = input( "Select GPU [0 or 1]: " )

if gpu_id in [ "0", "1" ]:
    os.environ[ "CUDA_VISIBLE_DEVICES" ] = gpu_id
else:
    print( "Invalid GPU id.  Defaulting to '0,1'" )

Select GPU [0 or 1]: 0


## Choose CPU Cores

In [2]:
cores = 12
share_cores = input( "Share CPU cores w/ other models? [y/n]: " )

if share_cores == "y":
    
    cores = cores / 2

print( "Allocating %d cores to this notebook" % cores )

# From: https://stackoverflow.com/questions/46421258/limit-number-of-cores-used-in-keras

from keras import backend as K
K.set_session(
    K.tf.Session(
        config=K.tf.ConfigProto(
            intra_op_parallelism_threads=32, inter_op_parallelism_threads=32 
        )
    )
)

Share CPU cores w/ other models? [y/n]: y
Allocating 6 cores to this notebook


Using TensorFlow backend.


In [3]:
import numpy as np
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
import time
import datetime
from math import sqrt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score, r2_score

from math import sqrt

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold

from matplotlib import pyplot as plt
%matplotlib inline

# My local install of jupyter notebook has broken matplotlib notebook and inline image rendering. Fix later! 
# from matplotlib import figure
# f = figure.Figure( figsize=( 10, 6 ) )
# #plt.figure( figsize=( 10, 6 ) )

cores = 12

def get_time( output=True ):
    
    temp = time.time()
    if output:
        now = datetime.datetime.now()
        print( now.strftime( "%Y.%m.%d %H:%M" ) )
        
    return temp

foo = get_time()

def print_time( start_time, end_time, interval="seconds" ):
    
    if interval == "hours":
        print ( "Time to process: [%s] hours" % ( str( ( end_time - start_time ) / 60 / 60 ) ) )
    else:
        print ( "Time to process: [%s] seconds" % ( str( end_time - start_time ) ) )

print_time( 0, 1 )

seed = 42

2018.06.03 14:37
Time to process: [1] seconds


In [4]:
page_start = get_time()
#trips = pd.read_csv( "data/green-tripdata-2015-09-cleaned.csv" )
trips = pd.read_parquet( "data/green-tripdata-2015-09-cleaned-plus-experimental-features.parquet" )
print_time( page_start, get_time() )
trips.head()

2018.06.03 14:37
2018.06.03 14:37
Time to process: [3.1850333213806152] seconds


,vendorid,store_and_fwd_flag,ratecodeid,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,trip_distance,fare_amount,...,payment_type_tip_bin_125_enc,ratecode_tip_bin_125_enc,payment_type_tip_bin_126_enc,ratecode_tip_bin_126_enc,payment_type_tip_bin_127_enc,ratecode_tip_bin_127_enc,payment_type_tip_bin_128_enc,ratecode_tip_bin_128_enc,payment_type_tip_bin_129_enc,ratecode_tip_bin_129_enc
0,2,False,5,-73.979485,40.684956,-73.979431,40.685020,1,0.00,7.8,...,0.000086,0.000000,0.000006,0.000000,0.000013,0.000000,0.000016,0.000000,0.000019,0.000000
1,2,False,5,-74.010796,40.912216,-74.010780,40.912212,1,0.00,45.0,...,0.000086,0.000000,0.000006,0.000000,0.000013,0.000000,0.000016,0.000000,0.000019,0.000000
2,2,False,1,-73.921410,40.766708,-73.914413,40.764687,1,0.59,4.0,...,0.000086,0.000041,0.000006,0.000003,0.000013,0.000006,0.000016,0.000008,0.000019,0.000009
3,2,False,1,-73.921387,40.766678,-73.931427,40.771584,1,0.74,5.0,...,0.000000,0.000041,0.000000,0.000003,0.000000,0.000006,0.000000,0.000008,0.000000,0.000009
4,2,False,1,-73.955482,40.714046,-73.944412,40.714729,1,0.61,5.0,...,0.000000,0.000041,0.000000,0.000003,0.000000,0.000006,0.000000,0.000008,0.000000,0.000009


In [4]:
trips.shape

(1455831, 305)

In [5]:
# create train & test sets: omit tip or related values
#X_trips = trips.drop( columns=[ "tip_percent", "tip_amount", "total_amount" ] )
X_trips = trips.drop( columns=[ "tip_percent", "tip_amount", "total_amount", "tip_recorded", "tip_percent_bin" ] )
y_trips = trips.tip_percent

# X_train, X_test, y_train, y_test = train_test_split( X_trips, y_trips, random_state=42 )
# eval_set = [ ( X_test, y_test ) ]

In [6]:
feature_count = X_trips.shape[ 1 ]
feature_count

300

In [5]:
# https://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/

def baseline_model():
    
    # create model
    model = Sequential()
    model.add( Dense( feature_count, input_dim=feature_count, kernel_initializer='normal', activation='relu' ) )
    model.add( Dense( 1, kernel_initializer='normal' ) )
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam' )
    
    return model

In [6]:

np.random.seed( seed )
# evaluate model with standardized dataset
estimator = KerasRegressor( build_fn=baseline_model, epochs=5, batch_size=512, verbose=True )

In [7]:
kfold = KFold( n_splits=5, random_state=seed )
results = cross_val_score( estimator, X_trips, y_trips, cv=kfold )
print( "Baseline results: %.2f (%.2f) MSE" % ( results.mean(), results.std() ) )

Epoch 1/5
1164664/1164664 [==============================] - 67s 58us/step - loss: 63.5557
Epoch 2/5
1164664/1164664 [==============================] - 67s 57us/step - loss: 54.4772
Epoch 3/5
1164664/1164664 [==============================] - 67s 57us/step - loss: 54.1084
Epoch 4/5
1164664/1164664 [==============================] - 67s 57us/step - loss: 53.8861
Epoch 5/5
291167/291167 [==============================] - 2s 6us/step
Epoch 1/5
1164665/1164665 [==============================] - 67s 58us/step - loss: 61.0054
Epoch 2/5
1164665/1164665 [==============================] - 67s 58us/step - loss: 54.7402
Epoch 3/5
1164665/1164665 [==============================] - 67s 58us/step - loss: 54.4325
Epoch 4/5
1164665/1164665 [==============================] - 67s 58us/step - loss: 54.0627
Epoch 5/5
291166/291166 [==============================] - 2s 6us/step
Epoch 1/5
1164665/1164665 [==============================] - 67s 58us/step - loss: 62.3578
Epoch 2/5
1164665/1164665 [============

In [8]:
print( "Baseline results: %.2f (%.2f) MSE" % ( results.mean(), results.std() ) )

Baseline results: -53.64 (0.68) MSE


In [18]:
# evaluate using standarized input values
np.random.seed( seed )
estimators = []
estimators.append( ( 'standardize', StandardScaler() ) )
estimators.append( ( 'mlp', KerasRegressor( build_fn=baseline_model, epochs=5, batch_size=512, verbose=True ) ) )
pipeline = Pipeline( estimators )
kfold = KFold( n_splits=5, random_state=seed )
results_scaled = cross_val_score( pipeline, X_trips, y_trips, cv=kfold )
print( "Standardized results: %.2f (%.2f) MSE" % ( results_scaled.mean(), results_scaled.std() ) )

Epoch 1/5
1164664/1164664 [==============================] - 4s 3us/step - loss: 62.9994
Epoch 2/5
1164664/1164664 [==============================] - 3s 3us/step - loss: 52.0922
Epoch 3/5
1164664/1164664 [==============================] - 3s 3us/step - loss: 51.8044
Epoch 4/5
1164664/1164664 [==============================] - 3s 3us/step - loss: 51.6528
Epoch 5/5
291167/291167 [==============================] - 1s 2us/step
Epoch 1/5
1164665/1164665 [==============================] - 4s 3us/step - loss: 63.3961
Epoch 2/5
1164665/1164665 [==============================] - 3s 3us/step - loss: 52.4526
Epoch 3/5
1164665/1164665 [==============================] - 3s 3us/step - loss: 52.1518
Epoch 4/5
1164665/1164665 [==============================] - 3s 3us/step - loss: 52.0119
Epoch 5/5
291166/291166 [==============================] - 1s 2us/step
Epoch 1/5
1164665/1164665 [==============================] - 4s 3us/step - loss: 62.4839
Epoch 2/5
1164665/1164665 [==============================

In [19]:
print( "Standardized results: %.2f (%.2f) MSE" % ( results_scaled.mean(), results_scaled.std() ) )
# print( "Standardized: mean [%.2f], std [%.2f], max [%.2f], and min[%.2f] RMSE" % ( np.sqrt( results_scaled.mean() ), np.sqrt( results_scaled.std() ), np.sqrt( results_scaled.max() ), np.sqrt( results_scaled.min() ) ) )

Standardized results: -51.74 (0.47) MSE


In [31]:
# create deeper model
def larger_model():
    
    model = Sequential()
    model.add( Dense( feature_count, input_dim=feature_count, kernel_initializer='normal', activation='relu' ) )
    # half as many nodes as features as above
    model.add( Dense( feature_count // 2, kernel_initializer='normal', activation='relu' ) )
    model.add( Dense( 1, kernel_initializer='normal' ) )
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam' )
    
    return model

In [33]:
np.random.seed(seed)
estimators = []
estimators.append( ( 'standardize', StandardScaler() ) )
estimators.append( ( 'mlp', KerasRegressor( build_fn=larger_model, epochs=5, batch_size=1000, verbose=True ) ) )
pipeline = Pipeline( estimators )
kfold = KFold( n_splits=5, random_state=seed )
results_larger = cross_val_score( pipeline, X_trips, y_trips, cv=kfold )
print( "Larger: %.2f (%.2f) MSE" % ( results_larger.mean(), results_larger.std() ) )

Epoch 1/5
1164664/1164664 [==============================] - 3s 2us/step - loss: 46.5948
Epoch 2/5
1164664/1164664 [==============================] - 2s 2us/step - loss: 24.9705
Epoch 3/5
1164664/1164664 [==============================] - 2s 2us/step - loss: 24.7440
Epoch 4/5
1164664/1164664 [==============================] - 3s 2us/step - loss: 24.6607
Epoch 5/5
291167/291167 [==============================] - 0s 1us/step
Epoch 1/5
1164665/1164665 [==============================] - 3s 2us/step - loss: 49.0822
Epoch 2/5
1164665/1164665 [==============================] - 2s 2us/step - loss: 25.2280
Epoch 3/5
1164665/1164665 [==============================] - 3s 2us/step - loss: 24.8287
Epoch 4/5
1164665/1164665 [==============================] - 2s 2us/step - loss: 24.6927
Epoch 5/5
291166/291166 [==============================] - 0s 1us/step
Epoch 1/5
1164665/1164665 [==============================] - 3s 2us/step - loss: 48.9477
Epoch 2/5
1164665/1164665 [==============================

In [34]:
# weird negative value!
#https://stackoverflow.com/questions/47346809/negative-result-for-regression-using-keras-and-tensorflow
print( "Larger: %.2f (%.2f) MSE" % ( results_larger.mean(), results_larger.std() ) )
# print( "Larger, Standardized: mean [%.2f], std [%.2f], max [%.2f], and min[%.2f] RMSE" % ( np.sqrt( results_larger.mean() ), np.sqrt( results_larger.std() ), np.sqrt( results_larger.max() ), np.sqrt( results_larger.min() ) ) )

Larger: -24.48 (0.55) MSE


In [44]:
np.sqrt( abs( results_larger.mean() ) )

4.9478546181160299

In [47]:
# define wider and deep model
def wider_model():
    
    model = Sequential()
    model.add( Dense( int( feature_count * 1.5 ), input_dim=feature_count, kernel_initializer='normal', activation='relu' ) )
    model.add( Dense( feature_count // 2, kernel_initializer='normal', activation='relu' ) )
    model.add( Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile( loss='mean_squared_error', optimizer='adam' )
    
    return model

In [53]:
start_time = get_time()

np.random.seed( seed )
estimators = []
estimators.append( ( 'standardize', StandardScaler() ) )
estimators.append( ( 'mlp', KerasRegressor( build_fn=wider_model, epochs=100, batch_size=1000, verbose=True ) ) )
pipeline = Pipeline( estimators )
kfold = KFold( n_splits=5, random_state=seed )
results_wider = cross_val_score( pipeline, X_trips, y_trips, cv=kfold )
print( "Wider: %.2f (%.2f) MSE" % ( results_wider.mean(), results_wider.std() ) )

print_time( start_time, get_time() )

2018.05.14 18:53
Epoch 1/100
1164664/1164664 [==============================] - 3s 2us/step - loss: 44.4229
Epoch 2/100
1164664/1164664 [==============================] - 3s 2us/step - loss: 24.8527
Epoch 3/100
1164664/1164664 [==============================] - 2s 2us/step - loss: 24.6543
Epoch 4/100
1164664/1164664 [==============================] - 3s 2us/step - loss: 24.5800
Epoch 5/100
1164664/1164664 [==============================] - 2s 2us/step - loss: 24.5404
Epoch 6/100
1164664/1164664 [==============================] - 2s 2us/step - loss: 24.5133
Epoch 7/100
1164664/1164664 [==============================] - 3s 2us/step - loss: 24.4822
Epoch 8/100
1164664/1164664 [==============================] - 3s 2us/step - loss: 24.4655
Epoch 9/100
1164664/1164664 [==============================] - 3s 2us/step - loss: 24.4608
Epoch 10/100
1164664/1164664 [==============================] - 3s 2us/step - loss: 24.4488
Epoch 11/100
1164664/1164664 [==============================] - 2s 2us/s

Epoch 90/100
1164664/1164664 [==============================] - 3s 2us/step - loss: 24.2135
Epoch 91/100
1164664/1164664 [==============================] - 3s 2us/step - loss: 24.2071
Epoch 92/100
1164664/1164664 [==============================] - 2s 2us/step - loss: 24.2037
Epoch 93/100
1164664/1164664 [==============================] - 2s 2us/step - loss: 24.1972
Epoch 94/100
1164664/1164664 [==============================] - 3s 2us/step - loss: 24.1967
Epoch 95/100
1164664/1164664 [==============================] - 3s 2us/step - loss: 24.1954
Epoch 96/100
1164664/1164664 [==============================] - 3s 2us/step - loss: 24.1920
Epoch 97/100
1164664/1164664 [==============================] - 3s 2us/step - loss: 24.1912
Epoch 98/100
1164664/1164664 [==============================] - 2s 2us/step - loss: 24.1906
Epoch 99/100
1164664/1164664 [==============================] - 3s 2us/step - loss: 24.1862
Epoch 100/100
291167/291167 [==============================] - 0s 1us/step
Epoch

1164665/1164665 [==============================] - 3s 2us/step - loss: 24.1010
Epoch 67/100
1164665/1164665 [==============================] - 3s 2us/step - loss: 24.0834
Epoch 68/100
1164665/1164665 [==============================] - 3s 2us/step - loss: 24.1465
Epoch 69/100
1164665/1164665 [==============================] - 3s 2us/step - loss: 24.1242
Epoch 70/100
1164665/1164665 [==============================] - 3s 2us/step - loss: 24.0814
Epoch 71/100
1164665/1164665 [==============================] - 3s 2us/step - loss: 24.0862
Epoch 72/100
1164665/1164665 [==============================] - 3s 2us/step - loss: 24.0820
Epoch 73/100
1164665/1164665 [==============================] - 3s 2us/step - loss: 24.0852
Epoch 74/100
1164665/1164665 [==============================] - 3s 2us/step - loss: 24.1105
Epoch 75/100
1164665/1164665 [==============================] - 3s 2us/step - loss: 24.0923
Epoch 76/100
1164665/1164665 [==============================] - 3s 2us/step - loss: 24.0824
E

1164665/1164665 [==============================] - 3s 2us/step - loss: 24.0525
Epoch 43/100
1164665/1164665 [==============================] - 3s 2us/step - loss: 24.0530
Epoch 44/100
1164665/1164665 [==============================] - 3s 2us/step - loss: 24.0416
Epoch 45/100
1164665/1164665 [==============================] - 3s 2us/step - loss: 24.0439
Epoch 46/100
1164665/1164665 [==============================] - 3s 2us/step - loss: 24.0268
Epoch 47/100
1164665/1164665 [==============================] - 3s 2us/step - loss: 24.0237
Epoch 48/100
1164665/1164665 [==============================] - 3s 2us/step - loss: 24.0210
Epoch 49/100
1164665/1164665 [==============================] - 3s 2us/step - loss: 24.0182
Epoch 50/100
1164665/1164665 [==============================] - 3s 2us/step - loss: 24.0166
Epoch 51/100
1164665/1164665 [==============================] - 3s 2us/step - loss: 24.0079
Epoch 52/100
1164665/1164665 [==============================] - 3s 2us/step - loss: 24.0065
E

In [51]:
# original vales
np.sqrt( abs( results.mean() ) )

4.940723891780662

In [54]:
np.sqrt( abs( results_wider.mean() ) )

4.9263345413124737

## WIll They Tip?

In [7]:
# create train & test sets: omit any mention of tips other than y value: 'tip_recorded'
X_tips = trips.drop( columns=[ "tip_amount", "tip_percent", "total_amount", "tip_recorded", "tip_percent_bin" ] )
y_tips = trips.tip_recorded

In [8]:
feature_count = X_tips.shape[ 1 ]
feature_count

300

In [9]:
def create_baseline_classifier():
    
    model = Sequential()
    model.add( Dense( feature_count, input_dim=feature_count, kernel_initializer='normal', activation='relu' ) )
    model.add( Dense( 1, kernel_initializer='normal', activation='sigmoid' ) )
    model.compile( loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'] )
    
    return model

In [10]:
start_time = get_time()

# evaluate model with standardized dataset
estimator = KerasClassifier( build_fn=create_baseline_classifier, epochs=5, batch_size=512, verbose=True )
kfold = StratifiedKFold( n_splits=5, shuffle=True, random_state=seed )
results_baseline_classifier = cross_val_score( estimator, X_tips, y_tips, cv=kfold )
print( "Baseline: %.2f%% (%.2f%%)" % ( results_baseline_classifier.mean() * 100, results_baseline_classifier.std() * 100 ) )

print_time( start_time, get_time(), interval="minutes" )

# 2018.05.25 16:22
# Baseline: 93.57% (0.05%)
# Time to process: [29.06] minutes

2018.06.03 14:39
Epoch 1/5
 516608/1164664 [============>.................] - ETA: 1:11 - loss: 0.2461 - acc: 0.9216

KeyboardInterrupt: 

In [13]:
# evaluate baseline model with standardized dataset
np.random.seed( seed )
estimators = []
estimators.append( ( 'standardize', StandardScaler() ) )
estimators.append( ( 'mlp', KerasClassifier( build_fn=create_baseline_classifier, epochs=5, batch_size=512, verbose=True ) ) )
pipeline = Pipeline( estimators )
kfold = StratifiedKFold( n_splits=5, shuffle=True, random_state=seed )
results_standardized_classifier = cross_val_score( pipeline, X_tips, y_tips, cv=kfold )
print( "Standardized: %.2f%% (%.2f%%)" % ( results_standardized_classifier.mean() * 100, results_standardized_classifier.std() * 100 ) )

#Standardized: 93.60% (0.02%)

Epoch 1/5
1164664/1164664 [==============================] - 8s 7us/step - loss: 0.1762 - acc: 0.9353
Epoch 2/5
1164664/1164664 [==============================] - 7s 6us/step - loss: 0.1731 - acc: 0.9357
Epoch 3/5
1164664/1164664 [==============================] - 7s 6us/step - loss: 0.1722 - acc: 0.9360
Epoch 4/5
1164664/1164664 [==============================] - 7s 6us/step - loss: 0.1714 - acc: 0.9361
Epoch 5/5
291167/291167 [==============================] - 1s 3us/step
Epoch 1/5
1164664/1164664 [==============================] - 7s 6us/step - loss: 0.1758 - acc: 0.9355
Epoch 2/5
1164664/1164664 [==============================] - 7s 6us/step - loss: 0.1733 - acc: 0.9357
Epoch 3/5
1164664/1164664 [==============================] - 7s 6us/step - loss: 0.1726 - acc: 0.9359
Epoch 4/5
1164664/1164664 [==============================] - 7s 6us/step - loss: 0.1716 - acc: 0.9361
Epoch 5/5
291167/291167 [==============================] - 1s 5us/step
Epoch 1/5
1164665/1164665 [===============

In [14]:
# make less wide?!?
def create_smaller():

    model = Sequential()
    model.add( Dense( feature_count // 2, input_dim=feature_count, kernel_initializer='normal', activation='relu' ) )
    model.add( Dense( 1, kernel_initializer='normal', activation='sigmoid' ) )
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'] )
    
    return model

estimators = []
estimators.append( ( 'standardize', StandardScaler() ) )
estimators.append( ( 'mlp', KerasClassifier(build_fn=create_smaller, epochs=5, batch_size=512, verbose=True ) ) )
pipeline = Pipeline( estimators )
kfold = StratifiedKFold( n_splits=5, shuffle=True, random_state=seed )
results_less_wide_classifier = cross_val_score( pipeline, X_tips, y_tips, cv=kfold )
print("Smaller: %.2f%% (%.2f%%)" % ( results_less_wide_classifier.mean() * 100, results_less_wide_classifier.std() * 100 ) )


Epoch 1/5
1164664/1164664 [==============================] - 7s 6us/step - loss: 0.1774 - acc: 0.9355
Epoch 2/5
1164664/1164664 [==============================] - 7s 6us/step - loss: 0.1735 - acc: 0.9356
Epoch 3/5
1164664/1164664 [==============================] - 7s 6us/step - loss: 0.1730 - acc: 0.9357
Epoch 4/5
1164664/1164664 [==============================] - 7s 6us/step - loss: 0.1721 - acc: 0.9360
Epoch 5/5
291167/291167 [==============================] - 1s 4us/step
Epoch 1/5
1164664/1164664 [==============================] - 7s 6us/step - loss: 0.1772 - acc: 0.9356
Epoch 2/5
1164664/1164664 [==============================] - 7s 6us/step - loss: 0.1735 - acc: 0.9357
Epoch 3/5
1164664/1164664 [==============================] - 7s 6us/step - loss: 0.1727 - acc: 0.9360
Epoch 4/5
1164664/1164664 [==============================] - 7s 6us/step - loss: 0.1719 - acc: 0.9361
Epoch 5/5
291167/291167 [==============================] - 1s 4us/step
Epoch 1/5
1164665/1164665 [===============

## Strip out Tipped Fares from All Fares

In [22]:
tipped = trips[ trips.tip_recorded == True ]
tipped.shape

(587882, 32)

## Given Only Tips, Can We Create a More Accurate Model?

In [23]:
X_tipped = tipped.drop( columns=[ "tip_amount", "tip_percent", "tip_percent_bin", "total_amount", "tip_recorded" ] )
y_tipped = tipped.tip_percent
X_tipped.shape

(587882, 28)

In [24]:
feature_count = X_tipped.shape[ 1 ]
feature_count

28

In [26]:
def baseline_tips_model():
    
    model = Sequential()
    model.add( Dense( feature_count, input_dim=feature_count, kernel_initializer='normal', activation='relu' ) )
    model.add( Dense( 1, kernel_initializer='normal' ) )
    model.compile(loss='mean_squared_error', optimizer='adam' )
    
    return model

np.random.seed( seed )
estimator = KerasRegressor( build_fn=baseline_tips_model, epochs=10, batch_size=1000, verbose=True )
kfold = KFold( n_splits=5, random_state=seed )
results_tips_baseline = cross_val_score( estimator, X_tipped, y_tipped, cv=kfold )
print("Baseline Tips Only Results: %.2f (%.2f) MSE" % ( results_tips_baseline.mean(), results_tips_baseline.std() ) )

Epoch 1/10
470305/470305 [==============================] - 3s 7us/step - loss: 85.6216
Epoch 2/10
470305/470305 [==============================] - 3s 6us/step - loss: 62.9592
Epoch 3/10
470305/470305 [==============================] - 3s 6us/step - loss: 61.2690
Epoch 4/10
470305/470305 [==============================] - 3s 6us/step - loss: 60.7912
Epoch 5/10
470305/470305 [==============================] - 3s 6us/step - loss: 60.5781
Epoch 6/10
470305/470305 [==============================] - 3s 6us/step - loss: 60.4517
Epoch 7/10
470305/470305 [==============================] - 3s 6us/step - loss: 60.3558
Epoch 8/10
470305/470305 [==============================] - 3s 6us/step - loss: 60.3203
Epoch 9/10
470305/470305 [==============================] - 3s 6us/step - loss: 60.2281
Epoch 10/10
117577/117577 [==============================] - 0s 2us/step
Epoch 1/10
470305/470305 [==============================] - 3s 7us/step - loss: 89.9425
Epoch 2/10
470305/470305 [=====================

In [28]:
np.sqrt( abs( results_tips_baseline.mean() ) )
# 7.8082648153739367? Let's hope using the zero's from will_tip predictions will counter balance?

7.8082648153739367

## Create Multiclass Sequential Network

In [37]:
X_trips = trips.drop( columns=[ "tip_percent", "tip_amount", "total_amount", "tip_recorded", "tip_percent_bin" ] )
y_trips = trips.tip_percent_bin.astype( 'int32' )
#y_trips = y_trips.astype( 'int32' )
y_trips.head()

0    25
1     0
2    12
3     0
4     0
Name: tip_percent_bin, dtype: int32

In [42]:
features_count = X_trips.shape[ 1 ]
features_count
class_count = y_trips.max() + 1
class_count

130

In [60]:
# define baseline model
def baseline_model():
    
    model = Sequential()
    model.add( Dense( features_count * 2, input_dim=features_count, kernel_initializer='normal', activation='relu' ) )
    #model.add( Dropout( .25 ) )
    model.add( Dense( features_count, kernel_initializer='normal', activation='relu' ) )
    #model.add( Dropout( .25 ) )
    model.add( Dense( features_count // 2, kernel_initializer='normal', activation='relu' ) )
    model.add( Dense( class_count, activation='softmax' ) )
    
    # Compile model
    model.compile( loss='categorical_crossentropy', optimizer='adam', metrics=[ 'accuracy' ] )
    
    #print( model.summary() )
    
    return model

baseline_model().summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_76 (Dense)             (None, 340)               58140     
_________________________________________________________________
dense_77 (Dense)             (None, 170)               57970     
_________________________________________________________________
dense_78 (Dense)             (None, 85)                14535     
_________________________________________________________________
dense_79 (Dense)             (None, 130)               11180     
Total params: 141,825
Trainable params: 141,825
Non-trainable params: 0
_________________________________________________________________
None


In [61]:
estimator = KerasClassifier( build_fn=baseline_model, epochs=100, batch_size=512, verbose=1 )
#kfold = KFold( n_splits=5, shuffle=True, random_state=seed )
kfold = StratifiedKFold( n_splits=2, shuffle=True, random_state=seed )
results = cross_val_score( estimator, X_trips, y_trips, cv=kfold )
print("Baseline: %.2f%% (%.2f%%)" % ( results.mean() * 100, results.std() * 100 ) )

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_80 (Dense)             (None, 340)               58140     
_________________________________________________________________
dense_81 (Dense)             (None, 170)               57970     
_________________________________________________________________
dense_82 (Dense)             (None, 85)                14535     
_________________________________________________________________
dense_83 (Dense)             (None, 130)               11180     
Total params: 141,825
Trainable params: 141,825
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
727882/727882 [==============================] - 40s 55us/step - loss: 1.3408 - acc: 0.6615
Epoch 2/100
727882/727882 [==============================] - 40s 54us/step - loss: 1.1557 - acc: 0.7072
Epoch 3/100
727882/727882 [============================

727949/727949 [==============================] - 39s 54us/step - loss: 0.9842 - acc: 0.7348
Epoch 40/100
727949/727949 [==============================] - 39s 54us/step - loss: 0.9844 - acc: 0.7346
Epoch 41/100
727949/727949 [==============================] - 39s 54us/step - loss: 0.9820 - acc: 0.7349
Epoch 42/100
727949/727949 [==============================] - 39s 54us/step - loss: 0.9811 - acc: 0.7349
Epoch 43/100
727949/727949 [==============================] - 39s 54us/step - loss: 0.9803 - acc: 0.7352
Epoch 44/100
727949/727949 [==============================] - 39s 54us/step - loss: 0.9794 - acc: 0.7352
Epoch 45/100
727949/727949 [==============================] - 39s 54us/step - loss: 0.9785 - acc: 0.7354
Epoch 46/100
727949/727949 [==============================] - 39s 54us/step - loss: 0.9780 - acc: 0.7352
Epoch 47/100
727949/727949 [==============================] - 39s 54us/step - loss: 0.9798 - acc: 0.7349
Epoch 48/100
727949/727949 [==============================] - 39s 54